In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#LOAD AND EXPLORE THE DATA

In [3]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/K-NN/creditcard_2023.csv')

In [ ]:
data.head()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


In [ ]:
data["Class"].value_counts()


,count
Class,
0,284315
1,284315


#APPLICATION ON ORIGINAL DATA

##1- KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import time
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split


knn = KNeighborsClassifier(n_neighbors=1, algorithm='auto', weights='distance', n_jobs=-1)

X = data.drop(columns=['Class'])
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
param_grid = {'n_neighbors': range(1, 10)}
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

start_time = time.time()


print("Best k:", grid_search.best_params_['n_neighbors'])
best_k = grid_search.best_params_['n_neighbors']
knn = KNeighborsClassifier(n_neighbors=best_k, algorithm='auto', weights='distance', n_jobs=-1)
# Train the model using PCA transformed training data
knn.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Training time:", training_time, "seconds")
y_pred = knn.predict(X_test)
prediction_time = time.time() - end_time

print("Prediction time:", prediction_time, "seconds")
print("Accuracy:", accuracy_score(y_test, y_pred))

Best k: 9
Training time: 0.1011056900024414 seconds
Prediction time: 83.90408420562744 seconds


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [ ]:
from sklearn.metrics import accuracy_score
start_time = time.time()


print("Best k:", grid_search.best_params_['n_neighbors'])
best_k = grid_search.best_params_['n_neighbors']
knn = KNeighborsClassifier(n_neighbors=best_k, algorithm='auto', weights='distance', n_jobs=-1)
# Train the model using PCA transformed training data
knn.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Training time:", training_time, "seconds")
y_pred = knn.predict(X_test)
prediction_time = time.time() - end_time

print("Prediction time:", prediction_time, "seconds")
print("Accuracy:", accuracy_score(y_test, y_pred))

Best k: 9
Training time: 0.14573955535888672 seconds
Prediction time: 85.43781590461731 seconds
Accuracy: 0.9990093147858302


##2- ID3

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import time

tree = DecisionTreeClassifier(criterion="entropy", random_state=42)
start_time = time.time()
tree.fit(X_train, y_train)
end_time = time.time()
print("Training time:", end_time - start_time, "seconds")
# Predict on test data
start_time2 = time.time()
y_pred = tree.predict(X_test)
end_time2 = time.time()
print("Prediction time:", end_time2 - start_time2, "seconds")
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Training time: 12.630340814590454 seconds
Prediction time: 0.03480195999145508 seconds
Test Accuracy: 0.9995603467984454


#CLEANING DATA

In [4]:
from sklearn.utils import resample

# Separate the two classes
class_0 = data[data['Class'] == 0]
class_1 = data[data['Class'] == 1]

# Downsample both classes to 50,000 each
class_0_downsampled = resample(class_0, replace=False, n_samples=50000, random_state=42)
class_1_downsampled = resample(class_1, replace=False, n_samples=50000, random_state=42)

# Combine the downsampled data
balanced_data = pd.concat([class_0_downsampled, class_1_downsampled])

# Shuffle the dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# Save the new dataset (optional)
balanced_data.to_csv('/content/drive/MyDrive/K-NN/balanced_dataset.csv', index=False)

In [5]:
#check for categorical data
balanced_data["Class"].value_counts()

,count
Class,
1,50000
0,50000


In [ ]:
balanced_data

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,529424,-0.819043,1.085489,-1.149642,1.651523,-0.943622,-1.724520,-1.196888,0.506881,-1.684193,...,0.653327,0.093500,-0.077904,0.851543,0.485963,0.323318,2.429973,2.101002,11040.94,1
1,434415,-0.113376,0.338494,-0.514143,0.831147,-0.294062,0.048543,-0.574933,0.178710,-0.815598,...,0.196135,-0.284421,-0.355971,-1.321984,1.196027,0.341433,1.241565,1.172904,4014.98,1
2,254626,-0.078860,-0.287465,1.654901,-0.827786,0.224548,0.327087,0.577556,-0.108098,0.539084,...,-0.130284,-0.184988,0.032983,0.105477,-0.524358,-1.345610,0.226224,0.399387,19308.17,0
3,516891,-0.497789,0.407065,-0.596811,0.265953,-0.231272,0.228945,-0.102203,0.210084,-0.023193,...,0.202876,0.751832,-0.038512,0.709546,-0.232977,-1.001528,0.125444,0.177560,16271.87,1
4,310731,0.234099,0.267460,-0.546588,0.585920,0.035365,-0.334644,-0.292280,0.021257,-0.342648,...,0.011746,-0.723328,-0.260316,-1.384580,1.416915,-0.498039,0.913392,1.019731,13417.44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,244128,1.643561,-0.531575,0.441470,-0.380290,-0.028032,-0.102397,0.332378,-0.198473,1.211809,...,-0.152250,-0.207429,0.377346,0.968066,-0.787938,-1.672360,-0.191032,-0.146381,10780.11,0
99996,558334,0.888601,0.323796,-0.716467,1.073780,0.602463,-0.596573,0.035903,-0.063492,-0.745106,...,0.022632,-0.416487,-0.111710,0.080129,0.683228,0.423206,0.426807,0.633664,9149.32,1
99997,543450,0.928281,0.422714,-0.621372,0.838082,1.815664,-0.363139,0.834401,-0.476788,-0.484507,...,0.325668,-0.639764,0.259464,-1.702483,-1.604966,0.077851,-0.077842,0.135085,3008.81,1
99998,88007,-0.464132,-0.413635,1.346685,-0.519964,0.761561,1.011181,0.328170,-0.233920,0.668487,...,0.128321,-0.050081,-0.518144,-0.902604,-0.904975,0.605593,-0.243823,-1.700893,2872.33,0


In [ ]:
balanced_data.shape

(100000, 31)

#REMOVE LOW VARIANCE FEATURES

In [6]:
from sklearn.feature_selection import VarianceThreshold
import pandas as pd

# Set a threshold (e.g., remove features with variance < 0.01)
selector = VarianceThreshold(threshold=0.01)
X = balanced_data.drop(columns=['Class'])  # Drop target column
# Store the 'Class' column before removing it
y = balanced_data['Class']
X_reduced = selector.fit_transform(X)

# Keep selected feature names
selected_features = X.columns[selector.get_support()]
balanced_data = pd.DataFrame(X_reduced, columns=selected_features)
# Add the 'Class' column back to the DataFrame
balanced_data['Class'] = y.values

In [ ]:
balanced_data

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,529424.0,-0.819043,1.085489,-1.149642,1.651523,-0.943622,-1.724520,-1.196888,0.506881,-1.684193,...,0.653327,0.093500,-0.077904,0.851543,0.485963,0.323318,2.429973,2.101002,11040.94,1
1,434415.0,-0.113376,0.338494,-0.514143,0.831147,-0.294062,0.048543,-0.574933,0.178710,-0.815598,...,0.196135,-0.284421,-0.355971,-1.321984,1.196027,0.341433,1.241565,1.172904,4014.98,1
2,254626.0,-0.078860,-0.287465,1.654901,-0.827786,0.224548,0.327087,0.577556,-0.108098,0.539084,...,-0.130284,-0.184988,0.032983,0.105477,-0.524358,-1.345610,0.226224,0.399387,19308.17,0
3,516891.0,-0.497789,0.407065,-0.596811,0.265953,-0.231272,0.228945,-0.102203,0.210084,-0.023193,...,0.202876,0.751832,-0.038512,0.709546,-0.232977,-1.001528,0.125444,0.177560,16271.87,1
4,310731.0,0.234099,0.267460,-0.546588,0.585920,0.035365,-0.334644,-0.292280,0.021257,-0.342648,...,0.011746,-0.723328,-0.260316,-1.384580,1.416915,-0.498039,0.913392,1.019731,13417.44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,244128.0,1.643561,-0.531575,0.441470,-0.380290,-0.028032,-0.102397,0.332378,-0.198473,1.211809,...,-0.152250,-0.207429,0.377346,0.968066,-0.787938,-1.672360,-0.191032,-0.146381,10780.11,0
99996,558334.0,0.888601,0.323796,-0.716467,1.073780,0.602463,-0.596573,0.035903,-0.063492,-0.745106,...,0.022632,-0.416487,-0.111710,0.080129,0.683228,0.423206,0.426807,0.633664,9149.32,1
99997,543450.0,0.928281,0.422714,-0.621372,0.838082,1.815664,-0.363139,0.834401,-0.476788,-0.484507,...,0.325668,-0.639764,0.259464,-1.702483,-1.604966,0.077851,-0.077842,0.135085,3008.81,1
99998,88007.0,-0.464132,-0.413635,1.346685,-0.519964,0.761561,1.011181,0.328170,-0.233920,0.668487,...,0.128321,-0.050081,-0.518144,-0.902604,-0.904975,0.605593,-0.243823,-1.700893,2872.33,0


In [ ]:
balanced_data.shape

(100000, 31)

#REMOVE HIGHLY CORRELATED FEATURES

In [7]:
import numpy as np

# Compute correlation matrix
corr_matrix = balanced_data.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation > 0.85
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]

# Drop highly correlated features
balanced_data = balanced_data.drop(columns=to_drop)

# Add the 'Class' column back to the DataFrame
balanced_data['Class'] = y.values

In [ ]:
balanced_data.shape

(100000, 30)

#FEATURE SCALING

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_columns = balanced_data.select_dtypes(include=['float64', 'int64']).columns
balanced_data[numeric_columns] = scaler.fit_transform(balanced_data[numeric_columns])


In [ ]:
balanced_data

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,1.491295,-0.819409,1.088721,-1.153070,1.657514,-0.939856,-1.722118,-1.280696,0.503555,-1.686971,...,0.656635,0.095509,-0.075797,0.851229,0.489621,0.319874,2.466108,2.158786,-0.138727,1.0
1,0.912514,-0.113330,0.341248,-0.515661,0.833196,-0.296549,0.050455,-0.614721,0.175187,-0.819079,...,0.196161,-0.284000,-0.350561,-1.320447,1.198991,0.338001,1.257583,1.204766,-1.156252,1.0
2,-0.182734,-0.078794,-0.285111,1.659901,-0.833709,0.217070,0.328922,0.619337,-0.111793,0.534509,...,-0.132603,-0.184148,0.033774,0.105799,-0.519711,-1.350127,0.225054,0.409645,1.058562,-1.0
3,1.414945,-0.497968,0.409864,-0.598577,0.265286,-0.234363,0.230807,-0.108532,0.206580,-0.027313,...,0.202950,0.756609,-0.036872,0.709353,-0.228615,-1.005824,0.122568,0.181622,0.618835,1.0
4,0.159049,0.234349,0.270169,-0.548204,0.586791,0.029707,-0.332626,-0.312063,0.017640,-0.346510,...,0.010447,-0.724752,-0.256042,-1.382990,1.419663,-0.502011,0.923854,1.047315,0.205446,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.246687,1.644634,-0.529377,0.442824,-0.384062,-0.033079,-0.100444,0.356807,-0.202223,1.206691,...,-0.154726,-0.206684,0.374047,0.967653,-0.783034,-1.677087,-0.199265,-0.151366,-0.176502,-1.0
99996,1.667410,0.889232,0.326541,-0.718593,1.076995,0.591347,-0.594483,0.039347,-0.067161,-0.748643,...,0.021411,-0.416620,-0.109201,0.080472,0.686693,0.419826,0.429032,0.650465,-0.412679,1.0
99997,1.576739,0.928936,0.425523,-0.623212,0.840164,1.792870,-0.361114,0.894361,-0.480704,-0.488255,...,0.326623,-0.640836,0.257565,-1.700622,-1.599263,0.074249,-0.084160,0.137961,-1.301969,1.0
99998,-1.197753,-0.464291,-0.411362,1.350760,-0.524408,0.748914,1.012826,0.352301,-0.237691,0.663807,...,0.127859,-0.048675,-0.510808,-0.901425,-0.899957,0.602330,-0.252950,-1.749296,-1.321735,-1.0


#SPLITTING

In [9]:
X = balanced_data.drop(columns=['Class'])  # Replace with actual target column
y = balanced_data['Class']

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#PCA (Since the dataset is large)

In [11]:
from sklearn.decomposition import PCA

pca = PCA(n_components=25)  # Reduce to 25 features
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
X_pca = pca.transform(X)

#KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1, algorithm='auto', weights='distance', n_jobs=-1)


#Find Optimal k

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': range(1, 10)}
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_pca, y_train)

print("Best k:", grid_search.best_params_['n_neighbors'])

Best k: 1


#Training

In [ ]:
import time

start_time = time.time()



best_k = grid_search.best_params_['n_neighbors']
knn = KNeighborsClassifier(n_neighbors=best_k, algorithm='auto', weights='distance', n_jobs=-1)
# Train the model using PCA transformed training data
knn.fit(X_train_pca, y_train)

end_time = time.time()
print("Training time:", end_time - start_time, "seconds")

Training time: 0.004627704620361328 seconds


##Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = knn.predict(X_test_pca)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9985666666666667
              precision    recall  f1-score   support

        -1.0       1.00      1.00      1.00     15000
         1.0       1.00      1.00      1.00     15000

    accuracy                           1.00     30000
   macro avg       1.00      1.00      1.00     30000
weighted avg       1.00      1.00      1.00     30000



#ID3 IMPLEMENTATION

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


##Training

In [ ]:
import time

tree = DecisionTreeClassifier(criterion="entropy", random_state=42)
start_time = time.time()
tree.fit(X_train_pca, y_train)
end_time = time.time()
print("Training time:", end_time - start_time, "seconds")

Training time: 2.6010773181915283 seconds


##Evaluation

In [ ]:
# Predict on test data
y_pred = tree.predict(X_test_pca)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


Test Accuracy: 0.9933666666666666


#LEAVE ONE OUT CROSS VALIDATION (LOOCV)

##1- KNN (NOT FILTERED)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import time

start_time = time.time()
# Define KNN model (with k=5)
knn = KNeighborsClassifier(n_neighbors=5)

# Define LOOCV strategy
loo = LeaveOneOut()

# Compute LOOCV scores
loo_scores = cross_val_score(knn, X, y, cv=loo, scoring='accuracy')
end_time = time.time()
# Print LOOCV accuracy
print("Training time:", end_time - start_time, "seconds")
print("LOOCV Accuracy:", np.mean(loo_scores))


Training time: 4043.757572412491 seconds
LOOCV Accuracy: 0.99841


##2- ID3 (NOT FILTERED)

In [ ]:
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import time

# Split features and target
X =  balanced_data.drop(columns=['Class'])
y = balanced_data['Class']

start_time = time.time()
# Initialize model (ID3)
tree = DecisionTreeClassifier(criterion="entropy", random_state=42)

# Apply LOOCV
loo = LeaveOneOut()
scores = cross_val_score(tree, X, y, cv=loo, scoring='accuracy', n_jobs=-1)
end_time= time.time()
print("Training time:", end_time - start_time, "seconds")
# Print results
print("LOOCV Accuracy:", np.mean(scores))


##1- KNN FILTERED

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import time

start_time = time.time()
# Define KNN model (with k=5)
knn = KNeighborsClassifier(n_neighbors=5)

# Define LOOCV strategy
loo = LeaveOneOut()

# Compute LOOCV scores
loo_scores = cross_val_score(knn, X_pca, y, cv=loo, scoring='accuracy')
end_time = time.time()
# Print LOOCV accuracy
print("Training time:", end_time - start_time, "seconds")
print("LOOCV Accuracy:", np.mean(loo_scores))


Training time: 1297.7649087905884 seconds
LOOCV Accuracy: 0.99825


##2- Decision Tree FILTERED

In [ ]:
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import time

# Split features and target
X =  balanced_data.drop(columns=['Class'])
y = balanced_data['Class']

start_time = time.time()
# Initialize model (ID3)
tree = DecisionTreeClassifier(criterion="entropy", random_state=42)

# Apply LOOCV
loo = LeaveOneOut()
scores = cross_val_score(tree, X_pca, y, cv=loo, scoring='accuracy')
end_time= time.time()
print("Training time:", end_time - start_time, "seconds")
# Print results
print("LOOCV Accuracy:", np.mean(scores))


KeyboardInterrupt: 

#5 FOLD CROSS-VALIDATION

##1- K-NN NOT FILTERED

In [ ]:
import time
from sklearn.model_selection import KFold, cross_val_score
start_time = time.time()

knn = KNeighborsClassifier(n_neighbors=9, algorithm='auto', weights='distance', n_jobs=-1)
# Define 5-Fold CV
kf5 = KFold(n_splits=5, shuffle=True, random_state=42)

# Compute 5-Fold CV scores
kf5_scores = cross_val_score(knn, X, y, cv=kf5, scoring='accuracy')
end_time = time.time()
# Print 5-Fold accuracy
print("Training time:", end_time - start_time, "seconds")
print("5-Fold Accuracy:", np.mean(kf5_scores))

Training time: 10.9262535572052 seconds
5-Fold Accuracy: 0.99795


##2- ID3 NOT FILTERED

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import time

start_time = time.time()
tree = DecisionTreeClassifier(criterion="entropy", random_state=42)
# Define 5-Fold CV
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Compute cross-validation scores
scores = cross_val_score(tree, X, y, cv=kf, scoring='accuracy')
end_time= time.time()
# Print results

print("Training time:", end_time - start_time, "seconds")
print("5-Fold Accuracy:", np.mean(scores))

Training time: 8.341856956481934 seconds
5-Fold Accuracy: 0.99943


##1- K-NN FILTERED

In [ ]:
import time
from sklearn.model_selection import KFold, cross_val_score
start_time = time.time()
# Define 5-Fold CV
knn = KNeighborsClassifier(n_neighbors=1, algorithm='auto', weights='distance', n_jobs=-1)
kf5 = KFold(n_splits=5, shuffle=True, random_state=42)

# Compute 5-Fold CV scores
kf5_scores = cross_val_score(knn, X_pca, y, cv=kf5, scoring='accuracy')
end_time = time.time()
# Print 5-Fold accuracy
print("Training time:", end_time - start_time, "seconds")
print("5-Fold Accuracy:", np.mean(kf5_scores))

Training time: 10.281925439834595 seconds
5-Fold Accuracy: 0.9986900000000001


##2- Decision Tree FILTERED

In [ ]:
from sklearn.model_selection import KFold

start_time = time.time()
# Define 5-Fold CV
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Compute cross-validation scores
scores = cross_val_score(tree, X_pca, y, cv=kf, scoring='accuracy')
end_time= time.time()
# Print results

print("Training time:", end_time - start_time, "seconds")
print("5-Fold Accuracy:", np.mean(scores))

Training time: 15.71240496635437 seconds
5-Fold Accuracy: 0.99428


#10 FOLD CROSS-VALIDATION

pretty much same as 5 fold cross-validation

##1- K-NN NOT FILTERED

In [ ]:
start_time = time.time()
# Define 10-Fold CV
knn = KNeighborsClassifier(n_neighbors=9, algorithm='auto', weights='distance', n_jobs=-1)
kf10 = KFold(n_splits=10, shuffle=True, random_state=42)

# Compute 10-Fold CV scores
kf10_scores = cross_val_score(knn, X, y, cv=kf10, scoring='accuracy')
end_time = time.time()
# Print 10-Fold accuracy

print("Training time:", end_time - start_time, "seconds")
print("10-Fold Accuracy:", np.mean(kf10_scores))

Training time: 12.229136228561401 seconds
10-Fold Accuracy: 0.99806


##1- K-NN FILTERED

In [ ]:
start_time = time.time()
# Define 10-Fold CV
knn = KNeighborsClassifier(n_neighbors=1, algorithm='auto', weights='distance', n_jobs=-1)
kf10 = KFold(n_splits=10, shuffle=True, random_state=42)

# Compute 10-Fold CV scores
kf10_scores = cross_val_score(knn, X_pca, y, cv=kf10, scoring='accuracy')
end_time = time.time()
# Print 10-Fold accuracy

print("Training time:", end_time - start_time, "seconds")
print("10-Fold Accuracy:", np.mean(kf10_scores))


Training time: 12.28014326095581 seconds
10-Fold Accuracy: 0.9988699999999999


##2- Decision Tree NO FILTER

In [ ]:
from sklearn.model_selection import KFold

start_time = time.time()
# Define 10-Fold CV
kf10 = KFold(n_splits=10, shuffle=True, random_state=42)

# Compute cross-validation scores
scores_10 = cross_val_score(tree, X, y, cv=kf10, scoring='accuracy')
end_time= time.time()
# Print results

print("Training time:", end_time - start_time, "seconds")
print("10-Fold Accuracy:", np.mean(scores_10))


Training time: 18.732490301132202 seconds
10-Fold Accuracy: 0.99942


##4- Decision Tree FILTERED

In [ ]:
from sklearn.model_selection import KFold

start_time = time.time()
# Define 10-Fold CV
kf10 = KFold(n_splits=10, shuffle=True, random_state=42)

# Compute cross-validation scores
scores_10 = cross_val_score(tree, X_pca, y, cv=kf10, scoring='accuracy')
end_time= time.time()
# Print results

print("Training time:", end_time - start_time, "seconds")
print("10-Fold Accuracy:", np.mean(scores_10))


Training time: 35.906330585479736 seconds
10-Fold Accuracy: 0.99462
